# ***MODOAP - Detection d'illustrations dans les documents historiques - ENTRAINEMENT***

**Détection d'images dans les documents historiques**


**Script d'auto apprentissage**

Ce script permet d'entraîner l'algorithme Mask-RCNN à la segmentation d'objets sur ses propres données d'entraînement. 

Il requiert un corpus d'entraînement : un dossier sur un Google Drive composé de deux dossiers "train" et "val" contenant chacun les images et leur fichier d'annotation.

L'annotation du corpus peut être opérée grâce aux outils VIA (https://www.robots.ox.ac.uk/~vgg/software/via/) et Annotate.

Le script implémente la configuration décrite sur https://github.com/matterport/Mask_RCNN

**Ce script doit impérativement être lancé dans un environnement GPU : Runtime -> Change runtime type -> GPU**

## 0. Connexion à un compte Google Drive, création de l'architecture et installation des pré-requis

Nécessite de se connecter à son compte Google Drive et d'entrer un code de vérification.

Crée un dossier Outils_Modoap sur le Drive qui servira à stocker les poids générés lors de l'apprentissage.

In [ ]:
from google.colab import drive
import os

if not os.path.exists("/content/drive/My Drive"):
  drive.mount('/content/drive')
else : print("Le Drive est déjà monté")

if not os.path.exists("/content/drive/My Drive/Outils_Modoap/Detection_Illustrations"):
  os.makedirs('/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Poids')

%cd
if not os.path.exists("/root/Mask_RCNN"):

  !git clone --quiet https://github.com/matterport/Mask_RCNN.git
  %cd /root/Mask_RCNN
  !pip install -q PyDrive
  !pip install -r requirements.txt
  !python setup.py install
  !cp ~/Mask_RCNN/samples/balloon/balloon.py ./illustration.py
  !sed -i -- 's/balloon/illustration/g' illustration.py
  !sed -i -- 's/Balloon/Illustration/g' illustration.py

else : print("L'algorithme est déjà téléchargé")

%tensorflow_version 1.x
!pip install q keras==2.1.5
!pip install q keras==2.1.5

# 1. Préparation du corpus d'entraînement

- **Pour un corpus annoté avec VIA :**

Le dossier du corpus doit être constitué de cette architecture :

(le nom et le format des images sont libres)

(le fichier via_region_data.json est obtenu dans VIA par Annotation -> Export Annotations (as json)


```
  train
  img1.jpg
  img2.jpg
  ...
  img100.jpg
  via_region_data.json
val
  img101.jpg
  img102.jpg
  ...
  img120.jpg
  via_region_data.json
```

- **Pour un corpus annoté avec Annotate :**

Le dossier du corpus doit être constitué de cette architecture:

Le nom et le format des images sont libres

Le fichier annotate_region_data.csv est obtenu dans Annotate par View and Export results of annotation -> Annotations -> Export CSV -> Use Coma Separator

Testé avec Annotate 1.7



```
  train
  img1.jpg
  img2.jpg
  ...
  img100.jpg
  annotate_region_data.csv
val
  img101.jpg
  img102.jpg
  ...
  img120.jpg
  annotate_region_data.csv
```

**Spécification du corpus**

Entrer le chemin absolu vers le dossier contenant le corpus sur le drive. 
La racine du Google Drive est */content/drive/My Drive/*

Possibilité de copier/coller le chemin depuis la fenêtre de gauche : *Files -> clic droit sur un dossier -> Copy Path*

Exemple de chemin:

/content/drive/My Drive/Corpus/corpus_pour_detection/ 

Ou bien entrer "exemple" pour télécharger un corpus de démonstration.

In [ ]:
dossier_corpus = input("Entrer le chemin absolu du dossier contenant le corpus, ou taper \"exemple\" ")
if dossier_corpus == "exemple" :
  if not os.path.exists("/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection_illustration/"):
    os.makedirs('/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection_illustration/')
  %cd /content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection_illustration/
  !wget https://github.com/cyril521/modoap-seg/raw/master/Datasets/exemple_entrainement_droitsetlibertes/corpus_train_val.7z.001
  !wget https://github.com/cyril521/modoap-seg/raw/master/Datasets/exemple_entrainement_droitsetlibertes/corpus_train_val.7z.002
  !wget https://github.com/cyril521/modoap-seg/raw/master/Datasets/exemple_entrainement_droitsetlibertes/corpus_train_val.7z.003

  !7z x ./corpus_train_val.7z.001
  os.remove("/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection_illustration/corpus_train_val.7z.001")
  os.remove("/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection_illustration/corpus_train_val.7z.002")
  os.remove("/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection_illustration/corpus_train_val.7z.003")

  !7z x ./corpus_entrainement_DL.zip
  os.remove("/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection_illustration/corpus_entrainement_DL.zip")
  dossier_corpus = "/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection_illustration/"


Entrer le chemin absolu du dossier contenant le corpus, ou taper "exemple" /content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection_illustration/


**3.4 Traduction des annotations réalisées avec VIA** (nécessaire)

Rappel : les fichiers d'annotations VIA dans les dossiers "train" et "val" doivent tous deux être nommés "via_region_data.json"

Cette cellule transforme les annotations rectangulaires en polygones à 4 coordonnées, traitables par l'algorithme

In [ ]:
import os
import json


def traductionVIA(viajson):
  annotations = json.load(open(viajson))
  if "_via_settings" in annotations.keys() :
      annotations_new = {}
      for image in annotations["_via_img_metadata"]:
          valeur_image = annotations["_via_img_metadata"][image]
          nom = image
          annotations_new[nom] = valeur_image
          for i in range(len(annotations["_via_img_metadata"][image]["regions"])) :
              if annotations["_via_img_metadata"][image]["regions"][i]["shape_attributes"]["name"] == "rect":

                  x = annotations["_via_img_metadata"][image]["regions"][i]["shape_attributes"]["x"]
                  y = annotations["_via_img_metadata"][image]["regions"][i]["shape_attributes"]["y"]
                  width = annotations["_via_img_metadata"][image]["regions"][i]["shape_attributes"]["width"]
                  height = annotations["_via_img_metadata"][image]["regions"][i]["shape_attributes"]["height"]

                  annotations_new[image]["regions"][i]["shape_attributes"]["all_points_x"] = [x, x+width, x+width, x]
                  annotations_new[image]["regions"][i]["shape_attributes"]["all_points_y"] = [y, y, y+height, y+height]
                  annotations_new[image]["regions"][i]["shape_attributes"]["name"] = "polygon"
                  del annotations_new[image]["regions"][i]["shape_attributes"]["y"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["x"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["width"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["height"]

  else :
      annotations_new = annotations
      for image in annotations :
          for i in range(len(annotations[image]["regions"])) :
              if annotations[image]["regions"][i]["shape_attributes"]["name"] == "rect":
                  x = annotations[image]["regions"][i]["shape_attributes"]["x"]
                  y = annotations[image]["regions"][i]["shape_attributes"]["y"]
                  width = annotations[image]["regions"][i]["shape_attributes"]["width"]
                  height = annotations[image]["regions"][i]["shape_attributes"]["height"]

                  annotations_new[image]["regions"][i]["shape_attributes"]["all_points_x"] = [x, x+width, x+width, x]
                  annotations_new[image]["regions"][i]["shape_attributes"]["all_points_y"] = [y, y, y+height, y+height]
                  annotations_new[image]["regions"][i]["shape_attributes"]["name"] = "polygon"
                  del annotations_new[image]["regions"][i]["shape_attributes"]["y"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["x"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["width"]
                  del annotations_new[image]["regions"][i]["shape_attributes"]["height"]
              
  with open('via_region_data.json', 'w') as json_file:
    json.dump(annotations_new, json_file)

path = os.path.join(dossier_corpus, "train", "via_region_data.json")
traductionVIA(path)

path2 = os.path.join(dossier_corpus, "val", "via_region_data.json")
traductionVIA(path2)

**3.5 Traduction des annotations réalisées avec Annotate** (necessaire)

Rappel : les fichiers d'annotations Annotate dans les dossiers "train" et "val" doivent tous deux être nommés "annotate_region_data.csv"

In [ ]:
import os 
import pandas as pd

def traductionANNOTATE(annotations_csv):

  annotationz_new = {}
  annotationz = pd.read_csv(annotations_csv).sort_values('File')

  for index, row in annotationz.iterrows() :
    filename = row["File"]
    size = ""
    regions = {}
    x = int(row["X"])
    y = int(row["Y"])
    w = int(row["W"])
    h = int(row["H"])
    all_points_x = [x, x+w, x+w, x]
    all_points_y = [y, y, y+h, y+h]
      
    if filename not in annotationz_new.keys(): 
      annotationz_new[filename] = {"filename":filename, "size":size,"regions":{0:{"shape_attributes":{"name":"polygon", "all_points_x":all_points_x,"all_points_y":all_points_y}, "region_attributes":{}}}, "file_attributes":{}}
      
    else :
      i = max(annotationz_new[filename]["regions"].keys()) + 1
      annotationz_new[filename]["regions"][i] = {"shape_attributes":{"name":"polygon", "all_points_x":all_points_x,"all_points_y":all_points_y}, "region_attributes":{}}

  with open('via_region_data.json', 'w') as json_file:
    json.dump(annotationz_new, json_file)
      
path = os.path.join(dossier_corpus, "train", "annotate_region_data.csv")
traductionANNOTATE(path)

path2 = os.path.join(dossier_corpus, "val", "annotate_region_data.csv")
traductionANNOTATE(path2)

**4. Préparation et configuration de l'algorithme**

Nécessite de préciser le nombre d'époques pour l'apprentissage ( par défaut : 30 )

In [ ]:
%cd /root/Mask_RCNN/

import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob

epoques = input("Entrer le nombre d'époques pour l'entraînement (par défaut : 30) : ")

!sed -i -- 's/epochs=30/epochs=$epoques/g' illustration.py 

ROOT_DIR = os.path.abspath("/")

import warnings
warnings.filterwarnings("ignore")

sys.path.append(ROOT_DIR)  
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import illustration

IMAGE_DIR = dossier_corpus

config = illustration.IllustrationConfig()

class InferenceConfig(config.__class__):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

**5. Lancement de l'entraînement**

L'entraînement peut prendre plusieurs heures en fonction du nombre d'époques choisi. Le résultat est un fichier de poids .h5 sauvegardé dans */content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Poids* (seul le dernier fichier h5 généré sera réutilisé par la suite)

Le paramètre --weights= permet de spécifier les poids initiaux à utiliser pour l'entraînement. Les valeurs possibles sont COCO, IMAGENET ou "/chemin/vers/fichier.h5"

In [ ]:
 %cd $dossier_corpus
!python /root/Mask_RCNN/illustration.py train --dataset=./ --weights=COCO --log="/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Poids"